In [4]:
## Vamo a importar los datos

import numpy as np
import pandas as pd
temp = pd.read_csv('nodo2_periodo3.csv')#,parse_dates=True,index_col=0)
import math

temp.head()

y = temp.iloc[:,4]
display(y.head())
display(y.tail())

from statsmodels.tsa.stattools import pacf
import matplotlib.pyplot as plt
x_pacf = pacf(y, nlags=30, method='ols')
plt.plot(x_pacf)
plt.plot(len(x_pacf)*[0],'--')
plt.title(' Partial autocorrelation function for Monthly Sunspots')
plt.show()

ventana = 10
x = pd.DataFrame(index=range(len(y)))
for i in range(ventana):
    x[i] = y.shift(i+1)
display(x.head(6))
x = x.dropna()
display(x.head(6))
display(x.tail(6))

y = y[ventana:]
display(y.head())
display(y.tail())
display(x.head())
display(x.tail())

## Vamo a normalizar los datos

from sklearn import preprocessing
# scaler para x
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

# 5 columnas
x = np.array(x).reshape((len(x),ventana))
x = scaler.fit_transform(x)

# scaler para y
scaler_y = preprocessing.MinMaxScaler(feature_range=(0,1))
y = np.array(y).reshape((len(y),1))
y = scaler_y.fit_transform(y)

# Train and Test Sets

#Define la forma de entrada y salida de los datos, las variables usadas para predecir y la variable que se va a predecir

train_end = int(len(y)*.8)
x_train = x[0:train_end,]
display('x_train: ',x_train[0:5])
x_test = x[train_end+1:len(y),]
y_train = y[0:train_end]
y_test = y[train_end+1:len(y)]
display('y_train: ',y_train[0:5])
x_train = x_train.reshape(x_train.shape+(1,))
x_test = x_test.reshape(x_test.shape+(1,))
# Set de validación
y_valid = y_train [train_end-50:train_end]
x_valid = x_train [train_end-50:train_end]

print('Shape of x_train is ', x_train.shape)
print('Shape of x_test is ', x_test.shape)

# importa el modelo secuencial, permite apilar capas de manera lineal
from keras.models import Sequential
# importa la capa densa, esta es una capa de red neuronal completamente conectada regular
# con una función lineal de activación
from keras.layers import Dense, Activation
# Optimizador de descenso de gradiente estocástico
from keras.layers.recurrent import LSTM
from keras.callbacks import EarlyStopping
from keras . layers import Dropout

## Vamo a definir el modelo

# define una semilla para tener resultados repetibles
seed = 1161180
np.random.seed(seed)
# Define el modelo secuencial
fit1 = Sequential()
# Agrega una capa LSTM con cuatro neuronas, activación tanh, activación interoir hard_sgimoitde
# y forma de entrada (5,1)
fit1.add(LSTM(units=4,activation='relu',
              recurrent_activation='hard_sigmoid',
              input_shape=(ventana,1)))
# añade 5% drop out
fit1.add(Dropout(0.1))
#Agrega una capa densa con una slida y activación lineal
fit1.add(Dense(units=1, activation='linear'))

# Define el modelo secuencial
fit2 = Sequential()
# Agrega una capa LSTM con cuatro neuronas, activación tanh, activación interoir hard_sgimoitde
# y forma de entrada (5,1)
fit2.add(LSTM(units=4,activation='relu',
              recurrent_activation='hard_sigmoid',
              input_shape=(ventana,1)))
#Agrega 5% de drop out
fit2.add(Dropout(0.1))
#Agrega una capa densa con una slida y activación lineal
fit2.add(Dense(units=1, activation='linear'))

fit1.compile(loss='mean_absolute_error', optimizer='adam')
#Ajustar el modelo
history1 = fit1.fit(x_train, y_train, batch_size = 1,epochs=10,
                    validation_data=(x_valid,y_valid),
                    callbacks =[EarlyStopping(monitor ='val_loss',patience=100,
                                              verbose=0,mode='auto')],verbose=0)

fit2.compile(loss='mean_absolute_error', optimizer='adam')
#Ajustar el modelo
history2 = fit2.fit(x_train, y_train, batch_size = 1, epochs=10, shuffle=True,
                    validation_data=(x_valid,y_valid),
                    callbacks =[EarlyStopping(monitor ='val_loss',patience=100,
                                              verbose=0,mode='auto')],verbose=0)

fit3 = Sequential ()
# The batch_input_shape takes the batch size (1 in our example), number of attributes 
#(5 time lagged variables) and number of time steps (1 month forecast).
fit3.add(LSTM(units=4,stateful=True,batch_input_shape=(1,ventana,1),activation='relu',
              recurrent_activation ='hard_sigmoid'))
fit3.add(Dropout(0.1))
fit3.add(Dense(units=1,activation='linear'))
fit3.compile(loss='mean_absolute_error',optimizer='adam')

fit4 = Sequential ()
# The batch_input_shape takes the batch size (1 in our example), number of attributes 
#(5 time lagged variables) and number of time steps (1 month forecast).
fit4.add(LSTM(units=4,stateful=True,batch_input_shape=(1,ventana,1),activation='relu',
              recurrent_activation ='hard_sigmoid'))
#fit4.add(Dropout(0.1))
fit4.add(Dense(units=1,activation='linear'))
fit4.compile(loss='mean_absolute_error',optimizer='adam')

end_point =len(x_train)
start_point =end_point - 50

#The model has to be trained one epoch at a time with the
#state reset after each epoch

history3=[]
for i in range(len(x_train[start_point:end_point])):
    history3.append(fit3.fit(x_train[start_point:end_point],y_train[start_point:end_point],epochs=1,
             batch_size=1,shuffle=False,validation_data=(x_valid,y_valid),
                             callbacks =[EarlyStopping(monitor ='val_loss',patience=100,
                                                       verbose=0,mode='auto')],verbose=0))
    fit3.reset_states()
    
history4=[]
for i in range(len(x_train[start_point:end_point])):
    history4.append(fit4.fit(x_train[start_point:end_point],y_train[start_point:end_point],epochs=1,
             batch_size=1,shuffle=True, validation_data=(x_valid,y_valid),
                             callbacks =[EarlyStopping(monitor ='val_loss',patience=100,
                                                       verbose=0,mode='auto')],verbose=0))
    fit4.reset_states()

from keras.layers.recurrent import GRU

fit5 = Sequential ()
#return_sequeinces=False para trabajar con una sola objetivo
#Para trabajar con varios objetivos se deja en True
fit5.add(GRU(units=4,return_sequences= False,activation='relu',
             recurrent_activation='hard_sigmoid',input_shape=(ventana,1)))
fit1.add(Dropout(0.05))
fit5.add(Dense(units=1,activation='linear'))
fit5.compile(loss='mean_absolute_error',optimizer='adam')

history5 = fit5.fit(x_train,y_train,batch_size=1,epochs=10, validation_data=(x_valid,y_valid),
                    callbacks =[EarlyStopping(monitor ='val_loss',patience=100,
                                              verbose=0,mode='auto')],verbose=0)

#the
#model will forecast the next month based on the last 500 rolling
#months of data:

#The larger the batch size, the more memory you
#will need to run the model

display(fit1.summary())
display(fit2.summary())
display(fit3.summary())
display(fit4.summary())
display(fit5.summary())

# Evaluar el modelo
#Train and Test MSE


for i in range(1,6):
    print('fit%d'%i)
    f = vars()[('fit%s')%str(i)]
    tr = vars()[('score_train%s')%str(i)] = f.evaluate(x_train, y_train, batch_size=1)
    ts = vars()[('score_test%s')%str(i)] = f.evaluate(x_test, y_test, batch_size=1)
    print('in train%d MAE = '%i, round(tr,4))
    print('in test%d MAE = '%i, round(ts,4),'\n')

#And to convert the predictions back to their original scale,
#so we can view them individually:

from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

entreno = scaler_y.inverse_transform(np.array(y_test).reshape((len(y_test),1)))
plt.figure(figsize=[16,8])
plt.subplot(3,2,6)
plt.plot(np.rint(entreno),label='set de prueba',linewidth=2)
plt.title('Set de prueba')

for i in [1,2,5]:
    plt.subplot(3,2,i)
    f = vars()[('fit%s')%str(i)]
    p = f.predict(x_test)
    p = vars()[('pred%s')%str(i)] = scaler_y.inverse_transform(np.array(p).reshape((len(p),1)))
    p1 = mean_absolute_error(p, entreno)
    plt.plot(np.rint(p))
    plt.plot(abs(p-entreno),alpha=.3)
    plt.title('fit%d\n MAE = %.2f'%(i,p1))

for i in [3,4]:
    plt.subplot(3,2,i)
    f = vars()[('fit%s')%str(i)]
    p = f.predict(x_test,batch_size=1)
    p = vars()[('pred%s')%str(i)] = scaler_y.inverse_transform(np.array(p).reshape((len(p),1)))
    p1 = mean_absolute_error(p, entreno)
    plt.plot(np.rint(p))
    plt.title('fit%d\n MAE = %.2f'%(i,p1))
    plt.plot(abs(p-entreno),alpha=.3)
plt.tight_layout()

plt.figure(figsize=[16,8])
for i in range(1,6):
    plt.subplot(3,2,i)
    p = vars()[('pred%s')%str(i)]
    plt.scatter(np.rint(p),np.rint(entreno),alpha=.1)
    plt.title('fit%d'%i)
plt.tight_layout()

plt.figure(figsize=[16,4])
plt.subplot(1,3,1)
plt.plot(history1.history['loss'], 'r.-', label='pérdidas sin shuffle')
plt.plot(history2.history['loss'], 'b.-', label='pérdidas con shuffle')
plt.plot(history5.history['loss'], 'g.-', label='pérdidas con GRU')
plt.legend()

plt.subplot(1,3,2)
h3 = [x.history['loss'] for x in history3]
plt.plot(h3, 'r.-', label='pérdidas con estado sin shuffle')
plt.legend()

plt.subplot(1,3,3)
h4 = [x.history['loss'] for x in history4]
plt.plot(h4, 'b.-',label='pérdidas con estado con shuffle')
plt.legend()

fit1.save('lstm.h5')
fit2.save('lstms.h5')
fit3.save('lstm_s.h5')
fit4.save('lstm_ss.h5')
fit5.save('grus.h5')
from sklearn.externals import joblib 
joblib.dump(scaler_y, 'scaler.save')

#CAmbiamos todos los mse por mae :D
#al cuarto le quitamos la capa Dropout

SyntaxError: invalid syntax (<ipython-input-4-bf26b2c1a740>, line 271)